In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
from torchvision import transforms
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import umap
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import sys
import numpy as np
np.set_printoptions(suppress=True, linewidth=np.nan, precision=2, threshold=sys.maxsize)

class MyDataset(Dataset):
    def __init__(self, mydataset, mylabel):
        self.mydataset = mydataset
        self.mylabel = mylabel

    def __getitem__(self, index):
        image = self.mydataset[index, :]
        label = self.mylabel[index]
        
        return image, label

    def __len__(self):
        return len(self.mylabel)

# hyper parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 1e-3
batch_size = 16
epochs = 150

# dataset and transform
trainset = MNIST('./data', train=True, transform=transforms.ToTensor(), download=True)
testset = MNIST('./data', train=False, transform=transforms.ToTensor())

# construct train test data matrix
mnist_train_data = []
mnist_train_label = []
mnist_test_data = []
mnist_test_label = []
    
for image, label in trainset:    
    img = np.array(image.view(-1))
    mnist_train_data.append(img)
    mnist_train_label.append(label)
    
for image, label in testset:    
    img = np.array(image.view(-1))
    mnist_test_data.append(img)
    mnist_test_label.append(label)
    
mnist_train_data = np.vstack(mnist_train_data)
mnist_train_label = np.vstack(mnist_train_label).reshape(-1)
mnist_test_data = np.vstack(mnist_test_data)
mnist_test_label = np.vstack(mnist_test_label).reshape(-1)

print("train", mnist_train_data.shape, mnist_train_label.shape)
print("test", mnist_test_data.shape, mnist_test_label.shape)

# perform PCA on training data --------------------
pca = PCA(n_components=0.9)
mnist_train_data_reduced = pca.fit_transform(mnist_train_data)
mnist_test_data_reduced = pca.transform(mnist_test_data)
print(mnist_train_data_reduced.shape, mnist_test_data_reduced.shape)

# feed back to trainset and testset
trainset_reduced = MyDataset(mnist_train_data_reduced, mnist_train_label)
testset_reduced = MyDataset(mnist_test_data_reduced, mnist_test_label)

# make a data loader
train_loader = DataLoader(trainset_reduced, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(testset_reduced, batch_size=batch_size, shuffle=False, num_workers=2)

train (60000, 784) (60000,)
test (10000, 784) (10000,)
(60000, 87) (10000, 87)


In [6]:
# build network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(87, 256, bias=False)
        self.fc2 = nn.Linear(256, 256, bias=False)
        self.fc3 = nn.Linear(256, 10, bias=False)

    def forward(self, x):
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# optimizer
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

print(model)


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} \t Loss: {:.6f}'.format(epoch, loss.item()))
     
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Net(
  (fc1): Linear(in_features=87, out_features=256, bias=False)
  (fc2): Linear(in_features=256, out_features=256, bias=False)
  (fc3): Linear(in_features=256, out_features=10, bias=False)
)


In [3]:
for epoch in range(1, epochs + 1):
    if epoch == 50:
        optimizer = optim.SGD(model.parameters(), lr=learning_rate*0.1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 	 Loss: 0.794467

Test set: Average loss: 0.9488, Accuracy: 8287/10000 (83%)

Train Epoch: 2 	 Loss: 1.019693

Test set: Average loss: 0.5108, Accuracy: 8832/10000 (88%)

Train Epoch: 3 	 Loss: 0.588864

Test set: Average loss: 0.3881, Accuracy: 8987/10000 (90%)

Train Epoch: 4 	 Loss: 0.362728

Test set: Average loss: 0.3310, Accuracy: 9077/10000 (91%)

Train Epoch: 5 	 Loss: 0.104373

Test set: Average loss: 0.2965, Accuracy: 9162/10000 (92%)

Train Epoch: 6 	 Loss: 0.213431

Test set: Average loss: 0.2718, Accuracy: 9230/10000 (92%)

Train Epoch: 7 	 Loss: 0.350352

Test set: Average loss: 0.2537, Accuracy: 9280/10000 (93%)

Train Epoch: 8 	 Loss: 0.290244

Test set: Average loss: 0.2387, Accuracy: 9319/10000 (93%)

Train Epoch: 9 	 Loss: 0.232737

Test set: Average loss: 0.2263, Accuracy: 9353/10000 (94%)

Train Epoch: 10 	 Loss: 0.072920

Test set: Average loss: 0.2157, Accuracy: 9379/10000 (94%)

Train Epoch: 11 	 Loss: 0.221285

Test set: Average loss: 0.2053, Acc


Test set: Average loss: 0.0893, Accuracy: 9725/10000 (97%)

Train Epoch: 89 	 Loss: 0.035763

Test set: Average loss: 0.0892, Accuracy: 9726/10000 (97%)

Train Epoch: 90 	 Loss: 0.055066

Test set: Average loss: 0.0892, Accuracy: 9725/10000 (97%)

Train Epoch: 91 	 Loss: 0.062608

Test set: Average loss: 0.0891, Accuracy: 9725/10000 (97%)

Train Epoch: 92 	 Loss: 0.027942

Test set: Average loss: 0.0890, Accuracy: 9726/10000 (97%)

Train Epoch: 93 	 Loss: 0.059180

Test set: Average loss: 0.0889, Accuracy: 9725/10000 (97%)

Train Epoch: 94 	 Loss: 0.014157

Test set: Average loss: 0.0888, Accuracy: 9725/10000 (97%)

Train Epoch: 95 	 Loss: 0.032421

Test set: Average loss: 0.0887, Accuracy: 9726/10000 (97%)

Train Epoch: 96 	 Loss: 0.042872

Test set: Average loss: 0.0887, Accuracy: 9726/10000 (97%)

Train Epoch: 97 	 Loss: 0.018170

Test set: Average loss: 0.0886, Accuracy: 9726/10000 (97%)

Train Epoch: 98 	 Loss: 0.038781

Test set: Average loss: 0.0885, Accuracy: 9726/10000 (97%)


In [7]:
for epoch in range(1, epochs + 1):
    if epoch == 50:
        optimizer = optim.SGD(model.parameters(), lr=learning_rate*0.1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 	 Loss: 1.727511

Test set: Average loss: 1.7184, Accuracy: 6413/10000 (64%)

Train Epoch: 2 	 Loss: 1.004948

Test set: Average loss: 0.8961, Accuracy: 8017/10000 (80%)

Train Epoch: 3 	 Loss: 0.818467

Test set: Average loss: 0.5738, Accuracy: 8596/10000 (86%)

Train Epoch: 4 	 Loss: 0.849011

Test set: Average loss: 0.4454, Accuracy: 8818/10000 (88%)

Train Epoch: 5 	 Loss: 0.253566

Test set: Average loss: 0.3785, Accuracy: 8957/10000 (90%)

Train Epoch: 6 	 Loss: 0.294846

Test set: Average loss: 0.3367, Accuracy: 9061/10000 (91%)

Train Epoch: 7 	 Loss: 0.198813

Test set: Average loss: 0.3075, Accuracy: 9117/10000 (91%)

Train Epoch: 8 	 Loss: 0.113400

Test set: Average loss: 0.2851, Accuracy: 9190/10000 (92%)

Train Epoch: 9 	 Loss: 0.163574

Test set: Average loss: 0.2668, Accuracy: 9242/10000 (92%)

Train Epoch: 10 	 Loss: 0.153630

Test set: Average loss: 0.2517, Accuracy: 9281/10000 (93%)

Train Epoch: 11 	 Loss: 0.211534

Test set: Average loss: 0.2392, Acc


Test set: Average loss: 0.1013, Accuracy: 9689/10000 (97%)

Train Epoch: 89 	 Loss: 0.035553

Test set: Average loss: 0.1012, Accuracy: 9690/10000 (97%)

Train Epoch: 90 	 Loss: 0.027280

Test set: Average loss: 0.1011, Accuracy: 9690/10000 (97%)

Train Epoch: 91 	 Loss: 0.021804

Test set: Average loss: 0.1010, Accuracy: 9691/10000 (97%)

Train Epoch: 92 	 Loss: 0.110302

Test set: Average loss: 0.1009, Accuracy: 9692/10000 (97%)

Train Epoch: 93 	 Loss: 0.134226

Test set: Average loss: 0.1008, Accuracy: 9692/10000 (97%)

Train Epoch: 94 	 Loss: 0.013576

Test set: Average loss: 0.1007, Accuracy: 9693/10000 (97%)

Train Epoch: 95 	 Loss: 0.115750

Test set: Average loss: 0.1007, Accuracy: 9695/10000 (97%)

Train Epoch: 96 	 Loss: 0.296929

Test set: Average loss: 0.1006, Accuracy: 9695/10000 (97%)

Train Epoch: 97 	 Loss: 0.277637

Test set: Average loss: 0.1004, Accuracy: 9695/10000 (97%)

Train Epoch: 98 	 Loss: 0.005726

Test set: Average loss: 0.1004, Accuracy: 9695/10000 (97%)


In [3]:
torch.save(model.state_dict(), "./model/mlp_model_1layer_1024")